# Tutorial: Batch Flight Requests
The `google_flights_scraper` package can also be used to make batch request of multiple flights where a certain criteria is varied (e.g. destination or dates)

## Set Up

In [1]:
# Import
from datetime import datetime, timedelta

from google_flights_scraper.jupyter_helper import (
    scrape_date_range,
    scrape_multiple_destinations,
)

# Get today's date
today = datetime.today()

## Destination Search
Search multiple destinations on certain dates. Helpful if you know when you want to travel, but don't know where and are looking for good deals. 


Below are the possible parameters when making a request:  

**Required Inputs**

`departure_code` : Departure airport or city (a single location)  
`departure_country` Departure country (a single country)  
`arrival_codes` : A list of possible destination airports or cities  
`arrival_countries` : The matching list of possible destination countries  
`start_date` : Departure date (in `"MM/DD/YYY"` format)  
`end_date` : Arrrival date  (in `"MM/DD/YYY"` format)  
`seat_classes` : The matching list of seat classes (reason this is `list` and not `str` is because domestic and international flights can have different class names)  

**Optional Inputs**  

`output_path` : path to output `.csv`  (ex: `export.csv`)  
`delay_seconds`: How many seconds to delay between requests. Can be helpful to increase if you are being rate limited by Google.    
`n_jobs`: Number of concurrent scrapes (default 1). When >1, `delay_seconds` is ignored and concurrency is controlled by the semaphore.  

### Example: Departing from New York

In this example, we are departing from New York and searching for flights to SEA, London, Paris, and SFO.

In [2]:
# Search Mapping
city_search = {
    "SEA": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
    "London": {"country": "United Kingdom", "seat_class": "Economy"},
    "Paris": {"country": "France", "seat_class": "Economy"},
    "SFO": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
}

# Individual Lists
destination_codes = list(city_search.keys())
destination_countries = [city_search[code]["country"] for code in destination_codes]
seat_classes = [city_search[code]["seat_class"] for code in destination_codes]

# Create Dates
start = (today + timedelta(weeks=8)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=10)).strftime("%m/%d/%Y")

# Run
df_destinations = scrape_multiple_destinations(
    departure_code="New York",
    departure_country="United States of America",
    arrival_codes=destination_codes,
    arrival_countries=destination_countries,
    start_date=start,
    end_date=end,
    seat_classes=seat_classes,
    delay_seconds=3.0,
    n_jobs=4,
)

In [3]:
df_destinations

,departure_airport,departure_country,arrival_airport,arrival_country,departure_date,return_date,seat_class,departure_airline,departure_departure_airport,departure_departure_date,...,return_duration_minutes,return_duration_str,return_carry_on_bags,return_checked_bags,price,price_classification,price_difference,price_relativity,status,url
0,New York,United States of America,Paris,France,04/07/2026,04/21/2026,Economy,American,John F. Kennedy International Airport,"Tuesday, April 7",...,730,12 hr 10 min,NaN,NaN,436,low,435,0.4994,Ran successfully.,https://www.google.com/travel/flights/booking?...
1,New York,United States of America,SFO,United States of America,04/07/2026,04/21/2026,Economy (exclude Basic),United,Newark Liberty International Airport,"Tuesday, April 7",...,337,5 hr 37 min,1.0,0.0,329,low,307,0.4827,Ran successfully.,https://www.google.com/travel/flights/booking?...
2,New York,United States of America,SEA,United States of America,04/07/2026,04/21/2026,Economy (exclude Basic),United,Newark Liberty International Airport,"Tuesday, April 7",...,321,5 hr 21 min,1.0,0.0,329,typical,0,0.0000,Ran successfully.,https://www.google.com/travel/flights/booking?...
3,New York,United States of America,London,United Kingdom,04/07/2026,04/21/2026,Economy,Norse Atlantic UK,John F. Kennedy International Airport,"Tuesday, April 7",...,470,7 hr 50 min,NaN,NaN,521,typical,0,0.0000,Ran successfully.,https://www.google.com/travel/flights/booking?...


This returns a pandas DataFrame of the best flight for each destination, sorted in order of `price_relativity` highlight the flights which are the "best deal".

This also contains all the other information returned by the basic flight request function.

## Date Range Search
Search all date combinations for a specific route between certain dates. Helpful if you're flexible on timing and looking for finding cheaper days to fly. 

Here are the possible parameters for this request:  

**Required Inputs**  

`departure_code` : Departure airport or city (a single location)  
`departure_country` : Departure country (a single country)  
`arrival_code` : Arrival airport or city (a single location)  
`arrival_country` : Arrival country (a single country)  
`start_date_range` : Start date for search (in `"MM/DD/YYY"` format)  
`end_date_range` : End date for search (in `"MM/DD/YYY"` format)  
`min_trip_length` : Minimum trip length (in days)  
`max_trip_length` : Maximum trip length (in days)  
`seat_class` : Class of seat (a single selection)  

**Optional Inputs**  

`output_path` : path to output `.csv`  (ex: `export.csv`)  
`delay_seconds`: How many seconds to delay between requests. Can be helpful to increase if you are being rate limited by Google.  
`n_jobs`: Number of concurrent scrapes (default 1). When >1, `delay_seconds` is ignored and concurrency is controlled by the semaphore.  

### Example: JFK to London
In this example, we are departing from JFK and arriving in London anytime within a two-week period, searching from trip from 5-7 days long.

In [4]:
# Create Dates
start = (today + timedelta(weeks=11)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=12)).strftime("%m/%d/%Y")

# Run
df_dates = scrape_date_range(
    departure_code="JFK",
    departure_country="United States of America",
    arrival_code="London",
    arrival_country="United Kingdom",
    start_date_range=start,
    end_date_range=end,
    min_trip_length=5,
    max_trip_length=7,
    seat_class="Economy",
    delay_seconds=3.0,
    n_jobs=6,
)

In [5]:
df_dates

,departure_airport,departure_country,arrival_airport,arrival_country,departure_date,return_date,seat_class,departure_airline,departure_departure_airport,departure_departure_date,...,return_duration_str,return_carry_on_bags,return_checked_bags,price,price_classification,price_difference,price_relativity,status,url,trip_length_days
0,JFK,United States of America,London,United Kingdom,04/30/2026,05/05/2026,Economy,British Airways,John F. Kennedy International Airport,"Thursday, April 30",...,7 hr 55 min,None,None,539,low,200,0.2706,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
1,JFK,United States of America,London,United Kingdom,04/29/2026,05/04/2026,Economy,American,John F. Kennedy International Airport,"Wednesday, April 29",...,7 hr 55 min,None,None,543,low,196,0.2652,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
2,JFK,United States of America,London,United Kingdom,04/28/2026,05/03/2026,Economy,British Airways,John F. Kennedy International Airport,"Tuesday, April 28",...,8 hr,None,None,543,low,150,0.2165,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
3,JFK,United States of America,London,United Kingdom,04/28/2026,05/05/2026,Economy,British Airways,John F. Kennedy International Airport,"Tuesday, April 28",...,7 hr 55 min,None,None,539,low,109,0.1682,Ran successfully.,https://www.google.com/travel/flights/booking?...,7
4,JFK,United States of America,London,United Kingdom,04/28/2026,05/04/2026,Economy,British Airways,John F. Kennedy International Airport,"Tuesday, April 28",...,7 hr 50 min,None,None,542,low,66,0.1086,Ran successfully.,https://www.google.com/travel/flights/booking?...,6
5,JFK,United States of America,London,United Kingdom,04/29/2026,05/05/2026,Economy,American,John F. Kennedy International Airport,"Wednesday, April 29",...,7 hr 50 min,None,None,542,low,66,0.1086,Ran successfully.,https://www.google.com/travel/flights/booking?...,6


This also returns a pandas DataFrame of the best flight for each destination, sorted in order of `price_relativity` highlight the flights which are the "best deal".

This also contains all the other information returned by the basic flight request function.